In [12]:
import pandas as pd
from modelling_prep import *
import os

In [ ]:
hnscc_clinical = pd.read_excel("/Users/katyscott/Documents/HNC Project/data/HNSCC/clinical_data/HNSCC-MDA-Data_update_20240514.xlsx")

In [ ]:
hnscc_clinical[hnscc_clinical['Follow up duration (month)'] > hnscc_clinical['Survival  (months)']]

## Add compactness to main radiomics file

In [8]:
datasetName = "HEAD-NECK-RADIOMICS-HN1"

In [9]:
radFileName = "radiomicfeatures_" + datasetName + ".csv"
compactnessRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features_compactness/", radFileName))

baseRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features/", radFileName))

print(len(baseRadiomics))
print(len(compactnessRadiomics))

137
137


In [10]:
negControls = ['shuffled_full', 'shuffled_roi', 'shuffled_non_roi', 'randomized_sampled_full', 'randomized_sampled_roi', 'randomized_sampled_non_roi']
#'randomized_full', 'randomized_roi', 'randomized_non_roi',

for negControl in negControls:
    radFileName =  "radiomicfeatures_" + negControl + "_" + datasetName + ".csv"

    compactnessRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features_compactness/", radFileName))

    baseRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features/", radFileName))
    print("# of patients in base", negControl, "radiomics:", len(baseRadiomics))
    print("# of patients in compactness", negControl, "radiomics:", len(compactnessRadiomics))

# of patients in base shuffled_full radiomics: 137
# of patients in compactness shuffled_full radiomics: 137
# of patients in base shuffled_roi radiomics: 137
# of patients in compactness shuffled_roi radiomics: 137
# of patients in base shuffled_non_roi radiomics: 137
# of patients in compactness shuffled_non_roi radiomics: 137
# of patients in base randomized_sampled_full radiomics: 137
# of patients in compactness randomized_sampled_full radiomics: 137
# of patients in base randomized_sampled_roi radiomics: 137
# of patients in compactness randomized_sampled_roi radiomics: 137
# of patients in base randomized_sampled_non_roi radiomics: 137
# of patients in compactness randomized_sampled_non_roi radiomics: 137


In [16]:
negControls = ['shuffled_full_', 'shuffled_roi_', 'shuffled_non_roi_', 'randomized_sampled_full_', 'randomized_sampled_roi_', 'randomized_sampled_non_roi_', '']
#'randomized_full', 'randomized_roi', 'randomized_non_roi',

for negControl in negControls:
    radFileName =  "radiomicfeatures_" + negControl + datasetName + ".csv"

    compactnessRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features_compactness/", radFileName))

    baseRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features/", radFileName))

    comRadID = getPatientIdentifierLabel(compactnessRadiomics)
    baseRadID = getPatientIdentifierLabel(baseRadiomics)

    compactnessRadiomics.set_index(comRadID, inplace=True)
    baseRadiomics.set_index(baseRadID, inplace=True)

    # Check that patient IDs match
    if all(compactnessRadiomics.index == baseRadiomics.index):

        compactOnly = compactnessRadiomics['original_shape_Compactness1']

        originalRadiomicsCols = baseRadiomics.filter(regex=r'^original_*')
        insertIndex = baseRadiomics.columns.get_loc(originalRadiomicsCols.columns[0])

        baseRadiomics.insert(insertIndex, 'original_shape_Compactness1', compactOnly)

        outputPath = os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "total_radiomic_features/", radFileName)

        if not os.path.exists(os.path.dirname(outputPath)):
            os.makedirs(os.path.dirname(outputPath))

        baseRadiomics.to_csv(outputPath)

    else:
        print("Mismatched patient IDs in compactness and base radiomics")


Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.


In [39]:
negControl = "shuffled_non_roi"
radFileName =  "radiomicfeatures_" + negControl + "_" + datasetName + ".csv"

compactnessRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features_compactness/", radFileName))

baseRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features/", radFileName))

In [40]:
comRadID = getPatientIdentifierLabel(compactnessRadiomics)
baseRadID = getPatientIdentifierLabel(baseRadiomics)

compactnessRadiomics.set_index(comRadID, inplace=True)
baseRadiomics.set_index(baseRadID, inplace=True)

# Check that patient IDs match
if all(compactnessRadiomics.index == baseRadiomics.index):

    compactOnly = compactnessRadiomics['original_shape_Compactness1']

    originalRadiomicsCols = baseRadiomics.filter(regex=r'^original_*')
    insertIndex = baseRadiomics.columns.get_loc(originalRadiomicsCols.columns[0])

    baseRadiomics.insert(insertIndex, 'original_shape_Compactness1', compactOnly)

else:
    print("Mismatched patient IDs in compactness and base radiomics")

Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.


In [41]:
outputPath = os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "total_radiomic_features/", radFileName)
baseRadiomics.to_csv(outputPath)

In [20]:
for idx in range(0, len(compactnessRadiomics)):
    if not (baseRadiomics.index[idx] == compactnessRadiomics.index[idx]):
        print(idx)
        print("Base rad value:", baseRadiomics.index[idx])
        print("Compact rad value:", compactnessRadiomics.index[idx])
        break

96
Base rad value: HNSCC-01-0320
Compact rad value: HNSCC-01-0319


In [1]:
import os

pyradiomicsParamFilePath = "/Users/katyscott/Documents/HNC Project/scripts/hnc_readii-2-roqc/config/default_pyradiomics.yaml"
newPyradiomicsParamFileName = "no_wavelet_" + os.path.basename(pyradiomicsParamFilePath)
newPyradiomicsParamFilePath = os.path.join(os.path.dirname(pyradiomicsParamFilePath), newPyradiomicsParamFileName)

In [12]:
import yaml
with open(pyradiomicsParamFilePath, 'r') as file:
    data = yaml.safe_load(file)

imageTypes = data.pop('imageType')
del imageTypes['Wavelet']
data['imageType'] = imageTypes

with open(newPyradiomicsParamFilePath, 'w') as newFile:
    yaml.safe_dump(data, newFile, default_flow_style=False)

In [9]:
del imageTypes['Wavelet']

In [6]:
data

{'featureClass': {'shape': None,
  'firstorder': None,
  'glcm': None,
  'glrlm': None,
  'glszm': None,
  'gldm': None,
  'ngtdm': None},
 'setting': {'binWidth': 25, 'resampledPixelSpacing': [1.0, 1.0, 1.0]},
 'imageType': {'Original': {},
  'Square': {},
  'SquareRoot': {},
  'Logarithm': {},
  'Exponential': {},
  'Gradient': {}}}

In [12]:
data['imageType'] = imageTypes

In [9]:
data['setting']['resampledPixelSpacing'] = [data['setting']['resampledPixelSpacing']] 

In [10]:
data

{'featureClass': {'shape': None,
  'firstorder': None,
  'glcm': None,
  'glrlm': None,
  'glszm': None,
  'gldm': None,
  'ngtdm': None},
 'setting': {'binWidth': 25, 'resampledPixelSpacing': [[1.0, 1.0, 1.0]]},
 'imageType': {'Original': {},
  'Square': {},
  'SquareRoot': {},
  'Logarithm': {},
  'Exponential': {},
  'Gradient': {}}}